In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer

In [2]:
df_jan = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [3]:
df_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [4]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [5]:
## Q1. Downloading the data

In [6]:
df_jan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [7]:
## Q2. Computing duration

In [8]:
df_jan["duration"] = (df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime).apply(lambda delta: delta.total_seconds() / 60)

In [9]:
round(df_jan.duration.std(), 2)

42.59

In [10]:
## Q3. Dropping outliers

In [11]:
before = len(df_jan)
before

3066766

In [12]:
df_jan = df_jan[(df_jan["duration"] >= 1) & (df_jan["duration"] <= 60)]

In [13]:
after = len(df_jan)
after

3009173

In [14]:
f"{100 * after / before:.0f}%"

'98%'

In [15]:
## Q4. One-hot encoding

In [16]:
features = ["PULocationID", "DOLocationID"]
dict_train = df_jan[features].astype(str).to_dict(orient="records")

encoder = DictVectorizer()
X_train = encoder.fit_transform(dict_train)

In [17]:
X_train.shape

(3009173, 515)

In [18]:
## Q5. Training a model. RMSE on train

In [19]:
from sklearn.linear_model import LinearRegression

In [20]:
lr_model = LinearRegression()
y_train = df_jan["duration"].values

lr_model.fit(X_train, y_train)

LinearRegression()

In [21]:
y_pred_train = lr_model.predict(X_train)

In [22]:
def rmse(y, y_pred):
    se = (y - y_pred) ** 2
    mse = se.mean()
    return round(np.sqrt(mse), 2)

In [23]:
rmse_train = rmse(y_train, y_pred_train)
rmse_train

7.65

In [24]:
## Q6. Evaluating the model. RMSE on validation

In [25]:
df_feb["duration"] = (df_feb.tpep_dropoff_datetime - df_feb.tpep_pickup_datetime).apply(lambda delta: delta.total_seconds() / 60)
df_feb = df_feb[(df_feb["duration"] >= 1) & (df_feb["duration"] <= 60)]

In [26]:
dict_val = df_feb[features].astype(str).to_dict(orient="records")

In [27]:
X_val = encoder.transform(dict_val)
y_val = df_feb.duration.values

In [28]:
X_val.shape

(2855951, 515)

In [29]:
y_pred_val = lr_model.predict(X_val)
rmse_val = rmse(y_val, y_pred_val)
rmse_val

7.81